# Topic Modeling (Gensim Library)


**Topic Modeling** - hier auch als “Themenanalyse” bezeichnet - beschreibt ein Verfahren des unüberwachten maschinellen Lernens, bei dem latente Themen auf Basis häufig gemeinsam auftretender Wörter explorativ identifiziert werden. Wir versuchen also durch das gemeinsame Auftreten von Wörtern auf latente Themen, die dieses Vorkommen erklären, zu schliessen.

## Ablauf...

* [Run Code](#1-bullet)
* [Tabelle mit Texten importieren](#2-bullet)
* [Den Text "reinigen" und Stop-Wörter entfernen](#3-bullet)
* [Lemmatisierung von Wörtern](#4-bullet)
* [Part of Speech Tagging ](#5-bullet)
* [Topic Modell trainieren](6#-bullet)
* [Topics visualisieren](#7-bullet)
* [Im Originaltext recherchieren](#8-bullet)


## Run Code <a class="anchor" id="1-bullet"></a>

Um Code in *Jupyter Notebooks* auszühren, müssen Sie auf **Run** klicken, das sich in der Menüleiste befindet. Klicken Sie dafür zunächst auf das Feld, das Sie ausführen möchten und anschließend auf **Run**. Sie können auch 'Shift' und 'Enter' verwenden. Wird das Feld gerade ausgeführt, erscheint links im Kästchen ein Sternchen. Dies bedeutet, dass das Programm gerade arbeitet. Wurde das Feld erfolgreich ausgeführt, erscheint eine Nummer im selben Kästchen. Gibt es einen Fehler, erscheint eine Fehlermeldung.

In [ ]:
#languange processing imports
import gensim
from gensim.models import LdaModel
from gensim import models, corpora, similarities
import gensim, spacy, logging, warnings
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import sys
import re, numpy as np, pandas as pd
from pprint import pprint
import spacy
from openpyxl import Workbook

#For the text overview
from nltk import FreqDist

#Word Cloud and Visualization

from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt

## Tabelle mit Texten importieren <a class="anchor" id="2-bullet"></a>

Hier sehen Sie die importierte Excel-Tabelle

In [ ]:
df = pd.read_excel(('Data/spanische_grippe.xlsx'), engine='openpyxl',) 
df.head()

## Den Text "reinigen" und Stop-Wörter entfernen <a class="anchor" id="3-bullet"></a>

Nun wird der Text gereinigt, das heißt, es werden Satzzeichen entfernt, alle Wörter werden einheitlich klein geschrieben und Stop-Wörter werden entfernt. Stop-Wörter sind Wörter, die häufig vorkommen und haben für gewöhnlich keine Relevanz für weitere Analysen. Hierzu gehören Wörter wie "und", "auf", "mit" usw. 

In [ ]:
#Functions to clean, tokenize, and stem the data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def initial_clean(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower() 
    text = nltk.word_tokenize(text, language = 'german')
    return text

stop_words = stopwords.words('german')#change the language here

stop_words.extend([]) #add additional words here
def remove_stop_words(text):
    return [word for word in text if word not in stop_words]

def apply_all(text):
    return remove_stop_words(initial_clean(text))
df['tokenized'] = df['Text'].apply(apply_all) 
df.head()

## Lemmatisierung und POS-Tagging <a class="anchor" id="5-bullet"></a>

In [ ]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'ADV', 'VERB']):
    texts_out = []
    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])#disable functions here
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

df['final'] = process_words(df['tokenized'])

## Topic Modell trainieren<a class="anchor" id="6-bullet"></a>


In [ ]:
def train_lda(data):
    num_topics = 20
    dictionary = corpora.Dictionary(data)
    corpus = [dictionary.doc2bow(doc) for doc in data]
    lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary,
                   minimum_probability=0.0, passes=20, iterations=200, per_word_topics=True)
    return dictionary,corpus,lda

dictionary,corpus,lda = train_lda(df['final'])

In [ ]:
lda.show_topic(topicid=1, topn=20)

## Topics visualisieren <a class="anchor" id="7-bullet"></a>

In [ ]:
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
vis

## Im Originaltext recherchieren <a class="anchor" id="8-bullet"></a>

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_text = df['Text']


lines_ = ['unit']
for line in df_text:
    if 'krank' in line: #insert here the word you are looking for
        lines_.append(line)
    
lines_ = pd.DataFrame(lines_, columns =['Text'])
df_select = pd.DataFrame(lines_['Text'])
df_select[0:10]    